In [4]:
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.layers import Dense, Dropout, LSTM, Embedding, Reshape, Input, InputLayer, Concatenate
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model

In [3]:
df = pd.read_csv('haikus.csv', encoding='latin1')
df = df[df['lang'] == 'en'].copy()
df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,haiku,lang
3,3,Memorial Day --,a shadow for each,white cross,NaN,NaN,NaN,NaN,NaN,NaN,Memorial Day --\r\na shadow for each\r\nwhite ...,en
4,4,spring rain -,as the doctor speaks,i think of lilacs,NaN,NaN,NaN,NaN,NaN,NaN,spring rain -\r\nas the doctor speaks\r\ni thi...,en
5,5,spring moonset --,a rice ball for,breakfast,NaN,NaN,NaN,NaN,NaN,NaN,spring moonset --\r\na rice ball for\r\nbreakfast,en
6,6,sunny afternoon,an old man lingers,near the mailbox,NaN,NaN,NaN,NaN,NaN,NaN,sunny afternoon\r\nan old man lingers\r\nnear ...,en
7,7,cinco de mayo,horses roll,in the shallows,NaN,NaN,NaN,NaN,NaN,NaN,cinco de mayo\r\nhorses roll\r\nin the shallows,en
8,8,quitting time,the smell of rain,in the lobby,NaN,NaN,NaN,NaN,NaN,NaN,quitting time\r\nthe smell of rain\r\nin the l...,en
9,9,waves,slowly cresting towards shore,a faint moon,NaN,NaN,NaN,NaN,NaN,NaN,waves\r\nslowly cresting towards shore\r\na fa...,en
10,10,overnight rain --,the scent of orange blossoms,in a desert town,NaN,NaN,NaN,NaN,NaN,NaN,overnight rain --\r\nthe scent of orange bloss...,en
13,13,misty summer rain,calling pheasant,in Zen temple,NaN,NaN,NaN,NaN,NaN,NaN,misty summer rain\r\ncalling pheasant\r\nin Ze...,en
14,14,day is done,poppies amidst,the dying grass,NaN,NaN,NaN,NaN,NaN,NaN,day is done\r\npoppies amidst\r\nthe dying grass,en


In [12]:
# 0 is start character, 60 is end

In [139]:
n_samples = 10000
samples = []
for i in range(n_samples):
    i = np.random.randint(1,50)
    samples.append(np.arange(i,i+9).reshape((3,3)))
samples = np.array(samples)
#inputs = np.append(np.zeros((n_samples,3,1), dtype=int), samples, axis=2)
inputs = np.append(samples[:,:,0].reshape((-1,3,1)), samples, axis=2)
outputs = np.append(samples, np.ones((n_samples,3,1), dtype=int)*60, axis=2)
X = np_utils.to_categorical(inputs, num_classes=n_tokens)
Y = np_utils.to_categorical(outputs, num_classes=n_tokens)
#samples

In [140]:
inputs

array([[[18, 18, 19, 20],
        [21, 21, 22, 23],
        [24, 24, 25, 26]],

       [[ 7,  7,  8,  9],
        [10, 10, 11, 12],
        [13, 13, 14, 15]],

       [[31, 31, 32, 33],
        [34, 34, 35, 36],
        [37, 37, 38, 39]],

       ..., 
       [[ 4,  4,  5,  6],
        [ 7,  7,  8,  9],
        [10, 10, 11, 12]],

       [[ 1,  1,  2,  3],
        [ 4,  4,  5,  6],
        [ 7,  7,  8,  9]],

       [[ 2,  2,  3,  4],
        [ 5,  5,  6,  7],
        [ 8,  8,  9, 10]]])

In [138]:
samples[:,:,0].reshape((-1,3,1))

array([[[22],
        [25],
        [28]],

       [[ 8],
        [11],
        [14]],

       [[47],
        [50],
        [53]],

       ..., 
       [[43],
        [46],
        [49]],

       [[23],
        [26],
        [29]],

       [[10],
        [13],
        [16]]])

In [112]:
n_tokens = 61
latent_dim = 256

# Keras Model

line0_in = Input(shape=(None, n_tokens))
line0_lstm_in = LSTM(latent_dim, return_state=True, return_sequences=True)
line0_out, line0_h, line0_c = line0_lstm_in(line0_in)

line0_lstm_out = LSTM(latent_dim, return_sequences=True)
line0_out = line0_lstm_out(line0_out, initial_state=[line0_h, line0_c])

line0_dense_out = Dense(n_tokens, activation='softmax')(line0_out)

model = Model(line0_in, line0_dense_out)
#model = Model(line0_in, line0_out)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [113]:
model.summary(line_length=150)

______________________________________________________________________________________________________________________________________________________
Layer (type)                                     Output Shape                     Param #           Connected to                                      
input_21 (InputLayer)                            (None, None, 61)                 0                                                                   
______________________________________________________________________________________________________________________________________________________
lstm_26 (LSTM)                                   [(None, None, 256), (None, 256), 325632            input_21[0][0]                                    
______________________________________________________________________________________________________________________________________________________
lstm_27 (LSTM)                                   (None, None, 256)                525312      

In [114]:
model.fit(X[:,0,:], Y[:,0,:], batch_size=64, epochs=4)

Epoch 1/4
10000/10000 [==============================] - 6s 594us/step - loss: 1.6596
Epoch 2/4
10000/10000 [==============================] - 4s 419us/step - loss: 0.1360
Epoch 3/4
10000/10000 [==============================] - 4s 416us/step - loss: 0.0079
Epoch 4/4
10000/10000 [==============================] - 4s 410us/step - loss: 0.0017


In [115]:
np.argmax(model.predict(X[:10,0,:]), axis=2)

array([[28, 29, 30, 60],
       [48, 49, 50, 60],
       [26, 27, 28, 60],
       [22, 23, 24, 60],
       [29, 30, 31, 60],
       [ 2,  3,  4, 60],
       [33, 34, 35, 60],
       [33, 34, 35, 60],
       [26, 27, 28, 60],
       [22, 23, 24, 60]], dtype=int64)

In [211]:
n_tokens = 61
latent_dim = 256

# Keras Model

line0_in = Input(shape=(None, n_tokens))#, batch_shape=(1,None,n_tokens))
#line0_lstm_in = LSTM(latent_dim, return_state=True, return_sequences=True, stateful=True)
line0_lstm_in = LSTM(latent_dim, return_state=True, return_sequences=True, stateful=False)
line0_out, line0_h, line0_c = line0_lstm_in(line0_in)

#line0_lstm_out = LSTM(latent_dim, return_sequences=True)
#line0_out = line0_lstm_out(line0_out, initial_state=[line0_h, line0_c])

line0_dense_out = Dense(n_tokens, activation='softmax')(line0_out)

line1_lstm_in = LSTM(latent_dim, return_state=True, return_sequences=True)
line1_out, line1_h, line1_c = line1_lstm_in(line0_out, initial_state=[line0_h, line0_c])
line1_dense_out = Dense(n_tokens, activation='softmax')(line1_out)

line2_lstm_in = LSTM(latent_dim, return_state=True, return_sequences=True)
line2_out, line2_h, line2_c = line2_lstm_in(line1_out, initial_state=[line1_h, line1_c])
line2_dense_out = Dense(n_tokens, activation='softmax')(line2_out)

model = Model(line0_in, [line0_dense_out, line1_dense_out, line2_dense_out])
#model = Model(line0_in, line0_out)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary(line_length=200)

________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                      Output Shape                                Param #                 Connected to                                                      
input_36 (InputLayer)                                             (None, None, 61)                            0                                                                                         
________________________________________________________________________________________________________________________________________________________________________________________________________
lstm_52 (LSTM)                                                    [(None, None, 256), (None, 256), (None, 256 325632                  input_36[0][0]                                                

In [213]:
model.fit(X[:,0,:], [Y[:,0,:], Y[:,1,:], Y[:,2,:]], batch_size=128, epochs=6)

Epoch 1/6
10000/10000 [==============================] - 6s 628us/step - loss: 1.3955 - dense_38_loss: 1.2150 - dense_39_loss: 0.1348 - dense_40_loss: 0.0457
Epoch 2/6
10000/10000 [==============================] - 6s 642us/step - loss: 0.7622 - dense_38_loss: 0.7321 - dense_39_loss: 0.0198 - dense_40_loss: 0.0103
Epoch 3/6
10000/10000 [==============================] - 6s 604us/step - loss: 0.3891 - dense_38_loss: 0.3826 - dense_39_loss: 0.0033 - dense_40_loss: 0.0033
Epoch 4/6
10000/10000 [==============================] - 6s 625us/step - loss: 0.1327 - dense_38_loss: 0.1322 - dense_39_loss: 3.8663e-04 - dense_40_loss: 6.8348e-05
Epoch 5/6
10000/10000 [==============================] - 7s 675us/step - loss: 0.0269 - dense_38_loss: 0.0269 - dense_39_loss: 3.1633e-05 - dense_40_loss: 5.3489e-06
Epoch 6/6
10000/10000 [==============================] - 7s 653us/step - loss: 0.0037 - dense_38_loss: 0.0037 - dense_39_loss: 4.2006e-06 - dense_40_loss: 1.2386e-06


In [228]:
np.argmax(model.predict(X[:1,0,:]), axis=3)

array([[[18, 19, 20, 60]],

       [[21, 22, 23, 60]],

       [[24, 25, 26, 60]]], dtype=int64)

In [126]:
np.array(model.predict(X[:10,0,:])).shape

(2, 10, 4, 61)

In [44]:
X[0]

array([[[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
  

In [47]:
X.shape

(100, 3, 4, 61)

In [49]:
X[:,0,:].shape

(100, 4, 61)

In [149]:
X[0].shape

(3, 4, 61)

In [161]:
np.argmax(model.predict(np.array([[X[0,0,0]]])), axis=3)

array([[[18]],

       [[21]],

       [[24]]], dtype=int64)

In [152]:
X[0,0,0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.], dtype=float32)

In [158]:
np.array([[X[0,0,0]]]).shape

(1, 1, 61)

In [238]:
#line0_in = Input(shape=(None, n_tokens))#, batch_shape=(1,None,n_tokens))
#line0_lstm_in = LSTM(latent_dim, return_state=True, return_sequences=True, stateful=True)
xline0_in = Input(shape=(None, n_tokens), batch_shape=(1,None,n_tokens))
xline0_lstm_in = LSTM(latent_dim, return_state=True, return_sequences=True, stateful=True)

xline0_out, xline0_h, xline0_c = xline0_lstm_in(xline0_in)

#line0_lstm_out = LSTM(latent_dim, return_sequences=True)
#line0_out = line0_lstm_out(line0_out, initial_state=[line0_h, line0_c])

xline0_dense_out = Dense(n_tokens, activation='softmax')(xline0_out)
model_line1 = Model(xline0_in, xline0_dense_out)

xline0_dense_out.set_weights(line0_dense_out.get_weights())
xline0_lstm_in.set_weights(line0_lstm_in.get_weights())

AttributeError: 'Tensor' object has no attribute 'set_weights'

In [242]:
model_line1_ = Model(line0_in, line0_dense_out)
model_line1 = Model(xline0_in, xline0_dense_out)
for nb,layer in enumerate(model_line1_.layers):
    model_line1.layers[nb].set_weights(layer.get_weights())
#model_line1_.layers

In [244]:
#np.argmax(model_line1.predict(np.array([[X[0,0,0]]])))
model_line1.reset_states()
line = [1]

def f():
    while line[-1] != 60:
        yield np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1))
    
#model_line1.predict(np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1)))
while line[-1] != 60:
    #line.append(np.argmax(model_line1.predict(np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1)))))
    line.append(np.argmax(model_line1.predict_generator(f(), steps=1)))
line

[1, 1, 2, 3, 60]

In [233]:
line

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [219]:
model_line1.predict_generator

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [188]:
np_utils.to_categorical(line[-1], num_classes=n_tokens).reshape((1,1,-1))

array([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]], dtype=float32)

In [191]:
line

[18, 60]